In [1]:
!pip install -q -U datasets
!pip install -q -U transformers
# !pip install -q -U rouge_score

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.1 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [2]:
!pip install accelerate -U -q
!pip install transformers[torch] -U -q

In [6]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_value_1 = user_secrets.get_secret("WANDB_API_KEY")

In [7]:
import torch 
import os
from huggingface_hub import login
import wandb
wandb.login(key=secret_value_1)
login(token=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset, load_metric


In [4]:
from datasets import Dataset
dataset_train = load_dataset("zqz979/meta-review", split="train")
dataset_eval = load_dataset("zqz979/meta-review", split="validation")
dataset_test = load_dataset("zqz979/meta-review", split="test")

def preprocess_dataset(dataset, input_column):
  def update_input(example):
    if example[input_column]:  # Check if empty
        if isinstance(example[input_column], str):  # Check if it's a string
            word_count = len(example["Output"].split())
            example[input_column] = f'summarize the following data in {word_count} words : {example[input_column]}'
        else:
            example[input_column] = "Data removed (not a string)"  # Placeholder for non-string data
            example["Output"] = "Output removed (input was not a string)"
    else:
        example[input_column] = "Data removed (empty)"  # Placeholder for empty data
        example["Output"] = "Output removed (input was empty)"
    return example

  return dataset.map(update_input)


filtered_dataset_train =preprocess_dataset(dataset_train, input_column="Input")
filtered_dataset_eval = preprocess_dataset(dataset_eval, input_column="Input")
filtered_dataset_test = preprocess_dataset(dataset_test, input_column="Input")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7692 [00:00<?, ? examples/s]

Map:   0%|          | 0/1648 [00:00<?, ? examples/s]

Map:   0%|          | 0/1649 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoTokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [10]:
max_input_length = 8192
max_output_length = 512
batch_size = 2

In [11]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["Input"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["Output"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [12]:
train_dataset = filtered_dataset_train.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["Input", "Output"],
)


val_dataset = filtered_dataset_eval.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["Input", "Output"],
)



Map:   0%|          | 0/7692 [00:00<?, ? examples/s]

Map:   0%|          | 0/1648 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForSeq2SeqLM

In [15]:
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

pytorch_model.bin:   0%|          | 0.00/648M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [16]:
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [ ]:
# rouge = load_metric("rouge")


In [17]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"]
    )["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [18]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

2024-03-12 16:12:17.037977: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 16:12:17.038113: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 16:12:17.164429: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [19]:
# enable fp16 apex training
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="./",
    logging_steps=5,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=0.1,
)

In [20]:
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    # compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=97, training_loss=3.2943158936254755, metrics={'train_runtime': 1532.712, 'train_samples_per_second': 0.502, 'train_steps_per_second': 0.063, 'total_flos': 4298726097027072.0, 'train_loss': 3.2943158936254755, 'epoch': 0.1})

In [23]:
led.save_pretrained("colab_model",safe_serialization=True)
tokenizer.save_pretrained("colab_model")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'min_length': 100, 'early_stopping': True, 'num_beams': 2, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. T

('colab_model/tokenizer_config.json',
 'colab_model/special_tokens_map.json',
 'colab_model/vocab.json',
 'colab_model/merges.txt',
 'colab_model/added_tokens.json',
 'colab_model/tokenizer.json')

In [ ]:
/kaggle/working/colab_model/config.json

In [26]:
led.push_to_hub("my-awesome-model", config="/kaggle/working/colab_model/config.json")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'min_length': 100, 'early_stopping': True, 'num_beams': 2, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. T

model.safetensors:   0%|          | 0.00/648M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Animus-09/my-awesome-model/commit/43d1e25572e8d0db0b06a28530533abee84c141a', commit_message='Upload LEDForConditionalGeneration', commit_description='', oid='43d1e25572e8d0db0b06a28530533abee84c141a', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
  encodeds = tokenizer(filtered_dataset_test["Input"][0], return_tensors="pt")

  model_inputs = encodeds.to("cuda:0")


  generated_ids = led.generate(**model_inputs)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Input ids are automatically padded from 2697 to 3072 to be a multiple of `config.attention_window`: 1024
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [31]:
print("human summary =====",filtered_dataset_test["Output"][0])
print("---------------------")
print("finetuned model =====" ,decoded)

human summary ===== The paper present results using syntactic information (primarily through constituency trees) on the task of recognizing argument discourse units. No reviewer recommends acceptance of the paper: - The empirical results appear strong, though the reviewers raise questions about some of the experimental choices.  - The writing is unclear and reviewers point out many missing or incorrect references in the bibliography. - There is little methodological novelty - known techniques are applied to a topic that has not been studied much. Overall, the area chair agrees with the reviewers that this work does not yet meet the bar for ICLR.
---------------------
finetuned model ===== This paper presents a new approach for addressing argument discourse unit (ADU) detection, by incorporating constituency trees in the model.  The paper presents an interesting approach to the problem of ADU recognition.   The reviewers agree that the paper is interesting, but the paper does not have a